In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [8]:
import os
import json
from datetime import datetime
from collections import Counter

import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from tqdm import tqdm
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [9]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name,
                 devices,
                 symptom):
        self.base_path = base_path
        self.folder_name = folder_name
        self.devices = devices
        self.symptom = symptom
        
        self.label_images()
      
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)

            label = 0 if data['metaData']['lesions'] == 'A7' else 1
            self.labeled_image_paths.append((image_path, label))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))

        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)
        image = self.transform(image)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 batch_size,
                 train_ratio=0,
                 num_workers=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, shuffle=False):
        dataloader = DataLoader(dataset=dataset,
                                batch_size=self.batch_size,
                                shuffle=shuffle,
                                num_workers=self.num_workers,
                                pin_memory=True)
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = self.make_dataloader(train_dataset, shuffle=True)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            labels = labels.type(torch.int)
            for label in labels.tolist():
                label_tuple = tuple(label)
                class_counts[label_tuple] = class_counts.get(label_tuple, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        self.start_time = datetime.now().strftime('%Y%m%d-%H%M%S')
        self.writer = SummaryWriter(log_dir=f'runs/{self.start_time}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(labels, predicted, average='binary')

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(labels, predicted)

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
    
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).int()
    
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar('LearningRate', self.scheduler.get_last_lr()[0], epoch)
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                probs = torch.sigmoid(outputs)
                predicted = (probs > 0.5).int()
        
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(np.array(all_predicted), np.array(all_labels))
        current_auc_roc = self.calculate_auc_roc(np.array(all_predicted), np.array(all_labels))
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save(self.model, f'{self.start_time}.pth')

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)

In [5]:
%%time
base_path = 'skin/Camera/Train'
folder_name = '일반'
devices = ['스마트폰', '일반카메라']
symptom = ['유', '상', '하', '초기', '비성숙', '성숙']

processor  = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                pet_type=pet_type,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

Total dataset size: 610329
Inspecting Data...
- Images shape: torch.Size([64, 3, 240, 240]), Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])
Inspecting Data...
- Images shape: torch.Size([64, 3, 240, 240]), Labels: tensor([0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0])


In [ ]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

In [ ]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              train_ratio=train_ratio)

In [6]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)
pos_weight = torch.tensor([processor.symptomatic_count / processor.asymptomatic_count], device=device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=data_loader.train_loader,
                       valid_dataloader=data_loader.test_loader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler)

In [ ]:
trainer.train(30)

Epoch 1/96:  28%|██▊       | 2397/8583 [14:34<36:27,  2.83batch/s]

In [10]:
%%time
base_path = 'skin/Valid'
folder_name = '일반'
devices = ['스마트폰', '일반카메라']
symptom = ['유', '상', '하', '초기', '비성숙', '성숙']

processor  = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

skin/Valid/반려견/피부/일반카메라
skin/Valid/반려묘/피부/일반카메라
Total images: 54223, Total JSON files: 54223
Total cases: 54223
Number of symptomatic cases: 28034, Number of asymptomatic cases: 26189
CPU times: user 2.3 s, sys: 631 ms, total: 2.93 s
Wall time: 2.93 s


In [11]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

CPU times: user 701 µs, sys: 0 ns, total: 701 µs
Wall time: 717 µs


In [15]:
%%time
batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

Inspecting Data...


- Class Counts:
  Class (1,): 26189 samples
  Class (0,): 28034 samples
CPU times: user 10.6 s, sys: 10 s, total: 20.7 s
Wall time: 3min 19s


In [23]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        loaded_model = torch.load(path)
        self.model.load_state_dict(loaded_model.state_dict())
        self.model.to(self.device)
        print("Loaded Model Information:")
        print(self.model)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [24]:
path = 'best_model.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

100%|██████████| 1695/1695 [02:47<00:00, 10.13it/s]


Evaluation Results:
Confusion Matrix:
[[24957  3077]
 [12881 13308]]
Accuracy: 0.7057
Precision: 0.8122
Recall: 0.5082
F1 Score: 0.6252
Min Probability: 0.5000
Max Probability: 0.9994
Standard Deviation of Probabilities: 0.1499
Mean Probability: 0.7343


In [25]:
path = 'best_model1.pth'
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

ModelTester(path=path, model=model, device=device, dataloader=data_loader.dataloader)

100%|██████████| 1695/1695 [02:45<00:00, 10.22it/s]


Evaluation Results:
Confusion Matrix:
[[ 9572 18462]
 [ 7232 18957]]
Accuracy: 0.5261
Precision: 0.5066
Recall: 0.7239
F1 Score: 0.5961
Min Probability: 0.5000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.1440
Mean Probability: 0.8749


In [26]:
path = 'best_model2.pth'
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

ModelTester(path=path, model=model, device=device, dataloader=data_loader.dataloader)

100%|██████████| 1695/1695 [02:46<00:00, 10.19it/s]


Evaluation Results:
Confusion Matrix:
[[12989 15045]
 [13021 13168]]
Accuracy: 0.4824
Precision: 0.4667
Recall: 0.5028
F1 Score: 0.4841
Min Probability: 0.5000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.1503
Mean Probability: 0.8254


In [ ]:
class ImageClassifier:
    def __init__(self, device, image_path, image_transform, model_weights_path, model):
        self.image_path = image_path
        self.image_transform = image_transform
        self.device = device
        self.model = model
        self.load_model_weights(model_weights_path)

    def preprocess_image(self):
        image = Image.open(self.image_path)
        input_data = self.image_transform(image).unsqueeze(0)
        input_data = input_data.to(self.device)
        return input_data

    def load_model_weights(self, weights_path):
        weights = torch.load(weights_path)
        self.model.load_state_dict(weights)
        self.model.to(self.device)

    def classify_image(self):
        self.model.eval()
        input_data = self.preprocess_image()

        with torch.no_grad():
            output = self.model(input_data)
        
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_prob, predicted_class = torch.max(probabilities, 1)
        
        class_label = "Disease" if predicted_class.item() == 0 else "Normal"
        class_prob = probabilities[0, predicted_class].item() * 100

        print(f"The image is predicted to be {class_label} with {class_prob:.2f}% probability.")
                
        return predicted_class.item(), class_prob

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_path = 'kIHjZsKQ6HdQ.jpg'
image_transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

model_weights_path = 'best_model.pth'
model = models.efficientnet_b1()
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

classifier = ImageClassifier(device=device,
                             image_path=image_path,
                             image_transform=image_transform,
                             model_weights_path=model_weights_path,
                             model=model)

predicted_class = classifier.classify_image()